In [1]:
import argparse
import time
import torch
from Models import get_model
from Process import *
import torch.nn.functional as F
from Optim import CosineWithRestarts
from Batch import create_masks
import pdb
import dill as pickle
import argparse
from Models import get_model
from Beam import beam_search
from nltk.corpus import wordnet
from torch.autograd import Variable
import re
from torch.utils.data import DataLoader

from spm_tokenize import *
from spm_vocab import *
from spm_handler import *

In [2]:
def multiple_replace(dict, text):
  # Create a regular expression  from the dictionary keys
  regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))

  # For each match, look-up corresponding value in dictionary
  return regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text) 

In [3]:
def convert_tokens_to_ids(vocab, tokenizer, sent):
    tokens = tokenizer.tokenizer(sent)
    token2idx = []

    for idx, token in enumerate(tokens):
        token2idx.append(vocab.stoi.get(token, vocab.unk_index))

    return token2idx

In [4]:
def translate_sentence(sentence, model, opt, vocab, tokenizer):
    
    model.eval()
    indexed = []
    #sentence = SRC.preprocess(sentence)
    print(sentence)
    sentence = convert_tokens_to_ids(vocab, tokenizer, sentence)

    sentence = Variable(torch.LongTensor([sentence]))
    if opt.device == 'cuda':
        sentence = sentence.cuda()
    
    sentence = beam_search(sentence, model, vocab, opt)

    return  multiple_replace({' ?' : '?',' !':'!',' .':'.','\' ':'\'',' ,':','}, sentence)

In [5]:
def translate(opt, model, vocab, tokenizer):
    sentences = opt.text.lower().split('.')
    translated = []

    for sentence in sentences:
        translated.append(translate_sentence(sentence + '.', model, opt, vocab, tokenizer).capitalize())

    return (' '.join(translated))

In [6]:
class Constant:
    def __init__(self):
        self.load_weights = 'checkpoint'
        self.k = 3
        self.max_len = 30
        self.d_model = 512
        self.n_layers = 6
        self.heads = 8
        self.dropout = 0.1
        self.no_cuda = 'store_true'
        self.floyd = 'store_true'

In [7]:
opt = Constant()

In [8]:
opt.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
print(f'Load Tokenizer and Vocab...')
sp_tokenizer = Tokenizer(is_train=False, model_prefix='spm')
sp_vocab = sp_tokenizer.vocab

Load Tokenizer and Vocab...


In [10]:
print(f'Load the extended vocab...')
vocab = Vocabulary.load_vocab('./data/vocab')

Load the extended vocab...


In [11]:
len(sp_vocab), len(vocab)

(32000, 32003)

In [12]:
######################TEST DATA######################
# fitting the test dataset dir
test_data_dir = ['./data/test/newstest2014_en', './data/test/newstest2014_de']
test_dataset = Our_Handler(src_path=test_data_dir[0], 
                        tgt_path=test_data_dir[1],
                        vocab=vocab, 
                        tokenizer=sp_tokenizer,
                        max_len=256,
                        is_test=True)

test_dataloader = DataLoader(test_dataset,
                        batch_size=8,
                        shuffle=False,
                        drop_last=True)
opt.test = test_dataloader
opt.test_len = len(test_dataloader)
####################################################

In [13]:
model = get_model(opt, len(sp_vocab), len(sp_vocab))

loading pretrained weights...


In [14]:
while True:
    #opt.text =input("Enter a sentence to translate (type 'f' to load from file, or 'q' to quit):\n")
    opt.text = 'I love you'
    if opt.text=="q":
        break
    if opt.text=='f':
        fpath =input("Enter a sentence to translate (type 'f' to load from file, or 'q' to quit):\n")
        try:
            opt.text = ' '.join(open(opt.text, encoding='utf-8').read().split('\n'))
        except:
            print("error opening or reading text file")
            continue
    phrase = translate(opt, model, vocab, sp_tokenizer)
    print('> '+ phrase + '\n')

i love you.


IndexError: index 0 is out of bounds for dimension 0 with size 0